# LMR Pseudo proxy experiment: Sea Ice 

Adapted from a notebook created by Greg Hakim  
Picked up by Katie Brennan  
May 2019  
October 2019 - add in 95th percentile calculations.   
June 2020 - adapted to work for common era reconstructions
July 2020 - adapted to use pseudo proxy data 
  
Goal: Draw temperature data from CCSM4 past1000 run at pages2kv2 locations and assimilate them into a 200 member ensemble drawn from CCSM4 past1000 run. Then the reconstruction can be compared to the True value in time (CCSM4 past1000 run). This will serve as an upper bound of how this method can perform in reconstructing Arctic sea ice in an idealized framework. 

In [1]:
import sys,os,copy
#sys.path.append("/Users/hakim/gitwork/LMR_python3")

In [2]:
import sys
import numpy as np
import pickle

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

from time import time
from spharm import Spharmt, getspecindx, regrid
from netCDF4 import Dataset
from scipy import stats

In [3]:
sys.path.insert(1,'/home/disk/kalman2/mkb22/LMR_lite/')
import LMR_lite_utils as LMRlite
import LMR_utils 
import LMR_config

sys.path.insert(1,'/home/disk/p/mkb22/Documents/si_utils_kb/')
import Sice_utils as siutils 

sys.path.insert(1,'/home/disk/p/mkb22/Documents/si_analysis_kb/common_era_experiments/')
import commonera_utils as ceutils 

Loading information from datasets.yml
Loading information from grid_def.yml


In [4]:
import importlib
# #importlib.reload(LMRlite)
# #importlib.reload(LMR_config)
# importlib.reload(LMRlite)
#importlib.reload(LMRlite)

In [37]:
def sub_arctic_plot(fig,ax,VAR1,LAT,LON,TITLE1,MAX1,colorbar=True,extent=True):
    var1, lon1 = add_cyclic_point(VAR1, coord=LON)
    new_lon2d, new_lat2d = np.meshgrid(lon1, LAT)
    if extent is True: 
        ax.set_extent([-150, 140, 50, 90], crs=ccrs.PlateCarree())
    ax.gridlines(linestyle='--')
    ax.add_feature(cfeature.LAND, facecolor=(1, 1, 1))
    cs = ax.pcolormesh(new_lon2d, new_lat2d, var1, 
                       vmin=-MAX1, vmax=MAX1, cmap=plt.cm.RdBu_r, 
                       transform=ccrs.PlateCarree())
    ax.coastlines(resolution='110m', linewidth=0.5)
    if colorbar is True:
        plt.colorbar(cs, ax=ax)
    ax.set_title(TITLE1)

In [38]:
proj = dict(projection=ccrs.Stereographic(central_latitude=90,
                                          central_longitude=-45,
                                          true_scale_latitude=0.1))

### User parameters: 

In [5]:
recon_start = 1400
recon_end = 1850

# inflate the sea ice variable here (can inflate whole state here too)
inflate = 1.0
inf_name = '1'
# inflate = 1.0
# inf_name = '1'

prior_name = 'ccsm4'

cfile = '/home/disk/kalman2/mkb22/LMR_lite/configs/config_ccsm4_brennan2020.yml'
#cfile = '/home/disk/kalman2/mkb22/LMR_lite/configs/config_ccsm4_fixedprox.yml'
#cfile = '/home/disk/kalman2/mkb22/LMR_lite/configs/config_production2.yml'

loc_list = [15000]

pp_err = 0.01

iteration = ['a']

savename = []
proxies = 'pseudo'
#proxies = 'fullLMRdbv0_3'

for t,it in enumerate(iteration):

    savedir = ('/home/disk/p/mkb22/nobackup/LMR_output/common_era_experiments/experiments/pseudo/')
        
#    savename = ['sic_'+prior_name+'_annual_recon_1850_2018_cru_R0_4_10deg_inf'+inf_name+'_loc'+str(loc_list[0])+it+'.pkl']#,
#  #               'sic_'+prior_name+'_annual_recon_1900_1940_cru_R0_4_10deg_inf'+inf_name+'_loc'+str(loc_list[1])+it+'.pkl']
    if it is 0: 
        savename = ['sic_'+prior_name+'_anrecon_'+str(recon_start)+'_'+
                    str(recon_end)+'_'+proxies+'_inf'+inf_name+'_loc'+str(loc_list[0])+'_'+it+'.pkl']
    else: 
        savename.append('sic_'+prior_name+'_anrecon_'+str(recon_start)+'_'+
                        str(recon_end)+'_'+proxies+'_inf'+inf_name+'_loc'+str(loc_list[0])+'_'+it+'.pkl')

In [40]:
savename

['sic_ccsm4_anrecon_1400_1850_pseudo_inf1_loc15000_a.pkl']

In [6]:
print('Starting experiment with inflation = '+str(inflate)+', '+inf_name)

print('loading configuration...')
cfg = LMRlite.load_config_simple(cfile)

Starting experiment with inflation = 1.0, 1
loading configuration...
Checking configuration ... 
OK!


In [13]:
print('loading prior')
X, Xb_one = LMRlite.load_prior(cfg)
Xbp = Xb_one - Xb_one.mean(axis=1,keepdims=True)

loading prior
('Reading file: ', '/home/disk/kalman3/rtardif/LMR/data/model/ccsm4_last_millenium/tas_sfc_Amon_CCSM4_past1000_085001-185012.nc')
(12012, 192, 288)
('indlat=', 0, ' indlon=', 1)
Anomalies provided as the prior: Removing the temporal mean (for every gridpoint)...
('tas', ': Global(monthly): mean=', 8.072375e-07, ' , std-dev=', 1.8899411)
('Averaging over month sequence:', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
('tas', ': Global(time-averaged): mean=', 4.4424884352419226e-08, ' , std-dev=', 0.8317386411161235)
('Reading file: ', '/home/disk/kalman3/rtardif/LMR/data/model/ccsm4_last_millenium/sic_sfc_OImon_CCSM4_past1000_085001-185012.nc')
(12012, 180, 360)
('indlat=', 0, ' indlon=', 1)
Full field provided as the prior
('sic', ': Global(monthly): mean=', 13.445376, ' , std-dev=', 32.317898)
('Averaging over month sequence:', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
('sic', ': Global(time-averaged): mean=', 13.445537533002385, ' , std-dev=', 31.06984518565762)
 
State vecto

In [14]:
prior_lat_full = X.prior_dict['tas_sfc_Amon']['lat']
prior_lon_full = X.prior_dict['tas_sfc_Amon']['lon']

prior_sic_lat_full = X.prior_dict['sic_sfc_OImon']['lat']
prior_sic_lon_full = X.prior_dict['sic_sfc_OImon']['lon']
prior_sic_lat_full.shape

(180, 360)

In [15]:
# use this for labeling graphics and output files
prior_id_string = X.prior_datadir.split('/')[-1]
print('prior string label: ',prior_id_string)

# check if config is set to regrid the prior
if cfg.prior.regrid_method:
    print('regridding prior...')
    # this function over-writes X, even if return is given a different name
    [X_regrid,Xb_one_new] = LMRlite.prior_regrid(cfg,X,Xb_one,verbose=True)
else:
    X_regrid.trunc_state_info = X.full_state_info

prior string label:  ccsm4_last_millenium
regridding prior...
0 55295
(55296, 200)
(55296, 2)
(120096, 2)
tas_sfc_Amon  : 2D lat/lon variable, truncating this variable
nlat,nlon: 192 288
=> Full array:      -9.281670570373535 8.779441833496094 -0.0068972888191832565 0.8136486978336512
=> Truncated array: -9.140680987365142 8.587566246732372 -0.006798206211061438 0.8089622241964994
55296 120095
(64800, 200)
(64800, 2)
(120096, 2)
sic_sfc_OImon  : 2D lat/lon variable, truncating this variable
nlat,nlon: 180 360
=> Full array:      0.0 99.99755859375 13.443183690710814 31.062699318206754
=> Truncated array: 0.0 99.77449605659926 13.454225861839634 30.52017673606255


In [16]:
X_regrid.coords.shape

(32760, 2)

In [17]:
# make a grid object for the prior
grid = LMRlite.Grid(X_regrid)

# locate 2m air temperature in Xb_one and make a new array
tas_pos = X_regrid.trunc_state_info['tas_sfc_Amon']['pos']
tas = Xb_one_new[tas_pos[0]:tas_pos[1]+1,:]

# fix using file system softlink for ccsm4 filename...
sic_pos = X_regrid.trunc_state_info['sic_sfc_OImon']['pos']
print('assigning '+prior_id_string+ ' sea ice ...')
sic = Xb_one_new[sic_pos[0]:sic_pos[1]+1,:]

assigning ccsm4_last_millenium sea ice ...


In [18]:
# Load full prior for tas and regrid
prior_tas_orig = X.prior_dict['tas_sfc_Amon']['value']

prior_regrid_tas_prep = np.transpose(np.reshape(prior_tas_orig,(prior_tas_orig.shape[0],
                                                                prior_tas_orig.shape[1]*prior_tas_orig.shape[2])))
prior_regrid_tas_prep.shape

nyears = 1001 

# Regrid full tas prior for proxy selection and verification: 
[prior_tas_regrid,lat_tas_new,lon_tas_new] = LMR_utils.regrid_esmpy(cfg.prior.esmpy_grid_def['nlat'],
                                                        cfg.prior.esmpy_grid_def['nlon'],
                                                        nyears,
                                                        prior_regrid_tas_prep,
                                                        prior_lat_full,
                                                        prior_lon_full,
                                                        prior_lat_full.shape[0],
                                                        prior_lat_full.shape[1],
                                                        method=cfg.prior.esmpy_interp_method)

prior_tas_regrid = np.reshape(np.transpose(prior_tas_regrid),(nyears,
                                                              cfg.prior.esmpy_grid_def['nlat'],
                                                              cfg.prior.esmpy_grid_def['nlon'],))

# # Load full prior for sic and regrid: 
# prior_sic_orig = X.prior_dict['sic_sfc_OImon']['value']

# prior_sic_regrid_prep = np.transpose(np.reshape(prior_sic_orig,
#                                                 (prior_sic_orig.shape[0],
#                                                  prior_sic_orig.shape[1]*prior_sic_orig.shape[2])))

# # Regrid full sic prior for proxy selection: 
# [prior_sic_regrid,lat_sic_new,lon_sic_new] = LMR_utils.regrid_esmpy(cfg.prior.esmpy_grid_def['nlat'],
#                                                               cfg.prior.esmpy_grid_def['nlon'],
#                                                               nyears,
#                                                               prior_sic_regrid_prep,
#                                                               prior_sic_lat_full,
#                                                               prior_sic_lon_full,
#                                                               180,360,
#                                                               method=cfg.prior.esmpy_interp_method)

# sic_prior_regrid = np.reshape(np.transpose(prior_sic_regrid),
#                               (nyears,
#                                cfg.prior.esmpy_grid_def['nlat'],
#                                cfg.prior.esmpy_grid_def['nlon'],))

In [19]:
# lalo_dir = '/home/disk/p/mkb22/nobackup/LMR_output/common_era_experiments/experiments/'
# lalo_savename = '20CR_grid.pkl'
# pickle.dump(grid,open(lalo_dir+lalo_savename, "wb"))

In [7]:
# load proxies
prox_manager = LMRlite.load_proxies(cfg)

ImportError: HDFStore requires PyTables, "No module named 'tables'" problem importing

In [27]:
for proxy_idx, Y in enumerate(prox_manager.sites_assim_proxy_objs()):
    dir(Y.error)

In [47]:
prox_lat = {}
prox_lon = {}
prox_err = {}
recon_years = range(1400,1850)

for yk, target_year in enumerate(recon_years):
    for proxy_idx, Y in enumerate(prox_manager.sites_assim_proxy_objs()):
        if target_year in Y.time: 
            if proxy_idx is 0:
                prox_lat[target_year] = []
                prox_lon[target_year] = []
                prox_err[target_year] = []
                prox_lat[target_year] = np.append(prox_lat[target_year],Y.lat)
                prox_lon[target_year] = np.append(prox_lon[target_year],Y.lon)
                prox_err[target_year] = np.append(prox_err[target_year],Y.psm_obj.R)
            else: 
                prox_lat[target_year] = np.append(prox_lat[target_year],Y.lat)
                prox_lon[target_year] = np.append(prox_lon[target_year],Y.lon)
                prox_err[target_year] = np.append(prox_err[target_year],Y.psm_obj.R)

In [46]:
for proxy_idx, Y in enumerate(prox_manager.sites_assim_proxy_objs()):
    print(Y.psm_obj.R)

0.07401781909069646
0.05917015433484852
0.054492613364871906
0.031494144208834604
0.12264914272913596
0.023037854403510846
0.18713087876086337
0.042931828507488036
0.05968660671770845
0.029613981901747726
0.5814210652890447
0.018500319269759903
0.026258896004886883
0.0732754268702685
0.03301443764545719
0.07596153882003634
0.018654119235337822
0.029293905956388342
0.0012620174096301304
0.025536878457800838
0.05558962870478201
0.04915990869027689
0.06546478283377558
0.1570952951786563
0.044971166698345985
0.013263661505766557
0.04834078565303595
0.7824999358607925
0.021666210871830977
0.036359479317096495
0.08537320236806442
0.061858561005447926
0.03958956235871814
0.05612628776048694
0.0010768842746272384
0.5388426232227952
0.09458753717273659
0.07563677175599912
0.06009760465518633
0.12674268828012397
0.0013309461365502168
0.09799567332228781
0.034484197315795814
0.04378944900779961
0.018063700459569662
0.04099133280980091
0.03866853565480466
0.1066540462369651
0.05174275673208894
0.0

In [49]:
prox_err[1400].shape

(136,)

In [49]:
target_year = 1200

# Get list of proxy locations for this year: 
prox_lat = {}
prox_lon = {}

for proxy_idx, Y in enumerate(prox_manager.sites_assim_proxy_objs()):
    if target_year in Y.time: 
        if proxy_idx is 0:
            prox_lat[target_year] = []
            prox_lon[target_year] = []
            prox_lat[target_year] = np.append(prox_lat[target_year],Y.lat)
            prox_lon[target_year] = np.append(prox_lon[target_year],Y.lon)
        else: 
            prox_lat[target_year] = np.append(prox_lat[target_year],Y.lat)
            prox_lon[target_year] = np.append(prox_lon[target_year],Y.lon)

In [59]:
recon_years = range(1400,1850)
prior_time = np.arange(850,1851,1)
target_year = 1400

ind = np.where(np.isclose(prior_time,target_year,atol=10e-5))
prior_time[ind]

array([1400])

In [21]:
# All proxies available given config setup: 
type_site_assim = prox_manager.all_ids_by_group
assim_proxy_count = len(prox_manager.all_proxies)
for pkey, plist in sorted(type_site_assim.items()):
    print(('%45s : %5d' % (pkey, len(plist))))
print(('%45s : %5d' % ('TOTAL', assim_proxy_count)))

                                 Bivalve_d18O :     1
               Corals and Sclerosponges_Rates :     2
                Corals and Sclerosponges_SrCa :    11
                Corals and Sclerosponges_d18O :    28
                        Ice Cores_MeltFeature :     1
                               Ice Cores_d18O :    28
                                 Ice Cores_dD :     7
                              Lake Cores_Misc :     2
                             Lake Cores_Varve :     5
                       Tree Rings_WidthPages2 :   343
                       Tree Rings_WoodDensity :    59
                                        TOTAL :   487


In [22]:
#--------------------------------------------------------
# start reconstruction experiments here for ALL reference datasets
#--------------------------------------------------------
# set parameters here

# NH surface area in M km^2 from concentration in percentage
nharea = 2*np.pi*(6380**2)/1e8

# inflate the entire state vector
if 2 == 1:
    print('inflating full state vector...')
    xbm = np.mean(Xb_one_new,1)
    xbp = Xb_one_new - xbm[:,None]
    Xb_one_inflate = np.copy(Xb_one_new)
    Xb_one_inflate = np.add(inflate*xbp,xbm[:,None])
else:
    # inflate sea ice only
    print('inflating only sea ice by '+str(inflate))
    xb_sicm = np.mean(Xb_one_new[sic_pos[0]:sic_pos[1]+1,:],1)
    xb_sicp = np.squeeze(Xb_one_new[sic_pos[0]:sic_pos[1]+1,:])-xb_sicm[:,None]
    sic_new = np.add(inflate*xb_sicp,xb_sicm[:,None])
    Xb_one_inflate = np.copy(Xb_one_new)
    Xb_one_inflate[sic_pos[0]:sic_pos[1]+1,:] = sic_new

inflating only sea ice by 1.0


In [23]:
print('loading Ye...')
[Ye_assim, 
Ye_assim_coords] = LMR_utils.load_precalculated_ye_vals_psm_per_proxy(cfg, 
                                                                      prox_manager,
                                                                      'assim',
                                                                      X.prior_sample_indices)

INFO  : Loading Ye file:ccsm4_last_millenium-pr_sfc_Amon-tas_sfc_Amon-anom_bilinear_seasonPSM-T:GISTEMP-PR:GPCC_ref1951-1980_cal1850-2015_LMRdbv1.1.0.npz
INFO  : Loading Ye file:ccsm4_last_millenium-tas_sfc_Amon-anom_linear_seasonPSM-GISTEMP_ref1951-1980_cal1850-2015_LMRdbv1.1.0.npz
INFO  : Now extracting proxy type-dependent Ye values...


loading Ye...


INFO  : Completed in 7.28355598449707secs


In [24]:
X_regrid.coords.shape

(32760, 2)

In [25]:
# nyear = 1000.0
# vY = []
# for proxy_idx, Y in enumerate(prox_manager.sites_assim_proxy_objs()):
#     try: 
#         vY.append(Y.values[nyear])
#     except KeyError: 
#         continue

In [26]:
# # Break up prior by two variables 
# Xb_one_tas = Xb_one[pp_pos[0]:pp_pos[1]+1,:]
# Xb_one_sic = Xb_one[pp_pos[1]+1:Xb_one.shape[0],:]

# # this is used for drawing prior ensemble estimates of pseudoproxy (Ye)
# Xb_sampler = np.reshape(Xb_one_tas,[grid.nlat,grid.nlon,grid.nens])
# print('pseduoproxy sampler shape:',Xb_sampler.shape)

In [29]:
# sic_dict_lalo = {}
# sic_ens_full = {}
# tas_dict_lalo = {}
# var_dict = {}
# obs_full = {}
# obs_size = {}
# Ye_full = {}
# sic_ens_dict = {}
# sie_ens_dict = {}

#cfg.core.loc_rad = loc
print('Localization radius: ',cfg.core.loc_rad)

#--------------------------------------------------------------------------------
# Loop over all years available in this reference dataset
recon_years = range(cfg.core.recon_period[0],cfg.core.recon_period[1])
#recon_years = range(1979,2000)
nyears = len(recon_years)

sic_save = []
nobs = []
sic_save_lalo = np.zeros((nyears,grid.nlat,grid.nlon))
tas_save_lalo = np.zeros((nyears,grid.nlat,grid.nlon))
sic_lalo_full = np.zeros((grid.nlat,grid.nlon,grid.nens))
var_save = []
sic_full_ens = []
sie_full_ens = []
sic_ndim = sic_pos[1]-sic_pos[0]+1
prox_lat = {}
prox_lon = {}

begin_time = time()
yk = -1
for yk, target_year in enumerate(recon_years):
    print('working on: '+ str(target_year))
    # Get relevant Ye's:
    [vY_real,vR_real,vP_real,vYe,
     vT,vYe_coords] = LMRlite.get_valid_proxies(cfg,prox_manager,target_year,
                                                Ye_assim,Ye_assim_coords, verbose=False)
    nobs = np.append(nobs,len(vY_real))
    print('Assimilating '+str(len(vY_real))+' proxy locations')
    
    prox_lat[yk] = vYe_coords[:,0]
    prox_lon[yk] = vYe_coords[:,1]
    vY = []
    vR = []
    
    for prox in range(len(vY_real)):
        # nearest prior grid lat,lon array indices for the proxy
        tmp = grid.lat[:,0]-np.array(prox_lat[yk])[prox]
        itlat = np.argmin(np.abs(tmp))
        tmp = grid.lon[0,:]-np.array(prox_lon[yk])[prox]
        itlon = np.argmin(np.abs(tmp))
        
        pseudoproxy = (prior_tas_regrid[yk,itlat,itlon] +                    
                       np.sqrt(pp_err)*np.random.randn(1))
        vY = np.append(vY,pseudoproxy)
        vR.append(pp_err)
    
    #xam,Xap,_ = LMRlite.Kalman_optimal(obs_QC,R_QC,Ye_QC,Xb_one_inflate)
    xam,Xap = LMRlite.Kalman_ESRF(cfg,vY,vR,vYe,
                                  Xb_one_inflate,X=X_regrid,
                                  vYe_coords=vYe_coords,verbose=False)

    tas_lalo = np.reshape(xam[tas_pos[0]:tas_pos[1]+1],[grid.nlat,grid.nlon])
    tas_save_lalo[yk,:,:] = tas_lalo

    # this saves sea-ice area for the entire ensemble
    sic_ens = []
    sie_ens = []
 #   for k in range(grid.nens):
    sic_lalo = np.reshape(xam[sic_pos[0]:sic_pos[1]+1,np.newaxis]+Xap[sic_pos[0]:sic_pos[1]+1,:],
                          [grid.nlat,grid.nlon,grid.nens])
    if 'full' in cfg.prior.state_variables['sic_sfc_OImon']:
        sic_lalo = np.where(sic_lalo<0.0,0.0,sic_lalo)
        sic_lalo = np.where(sic_lalo>100.0,100.0,sic_lalo)

        # Calculate extent: 
        sie_lalo = siutils.calc_sea_ice_extent(sic_lalo,15.0)
    else: 
        sic_lalo = sic_lalo
        sie_lalo = np.zeros(sic_lalo.shape)
            
    for k in range(grid.nens):
        _,nhmic,_ = LMR_utils.global_hemispheric_means(sic_lalo[:,:,k],grid.lat[:, 0])
        _,sie_nhmic,_ = LMR_utils.global_hemispheric_means(sie_lalo[:,:,k],grid.lat[:, 0])
        sic_ens.append(nhmic)
        sie_ens.append(sie_nhmic)

    sic_save_lalo[yk,:,:] = np.nanmean(sic_lalo,axis=2)
    var_save.append(np.var(sic_ens,ddof=1))
    sic_full_ens.append(sic_ens)
    sie_full_ens.append(sie_ens)

    # this saves the gridded concentration field for the entire ensemble
    #Xap_save[0,:,:] = Xap[sic_pos[0]:sic_pos[1]+1]
    #Xap_var = np.var(Xap[sic_pos[0]:sic_pos[1]+1,:],axis=1,ddof=1)
    #Xap_var_save[0,:] = Xap_var

    print('done reconstructing: ',target_year)

elapsed_time = time() - begin_time
print('-----------------------------------------------------')
print('completed in ' + str(elapsed_time) + ' seconds')
print('-----------------------------------------------------')

Localization radius:  15000
working on: 1400
Assimilating 143 proxy locations
done reconstructing:  1400
working on: 1401
Assimilating 144 proxy locations
done reconstructing:  1401
working on: 1402
Assimilating 143 proxy locations
done reconstructing:  1402
working on: 1403
Assimilating 143 proxy locations
done reconstructing:  1403
working on: 1404
Assimilating 144 proxy locations
done reconstructing:  1404
working on: 1405
Assimilating 143 proxy locations
done reconstructing:  1405
working on: 1406
Assimilating 144 proxy locations
done reconstructing:  1406
working on: 1407
Assimilating 145 proxy locations
done reconstructing:  1407
working on: 1408
Assimilating 143 proxy locations
done reconstructing:  1408
working on: 1409
Assimilating 144 proxy locations
done reconstructing:  1409
working on: 1410
Assimilating 145 proxy locations
done reconstructing:  1410
working on: 1411
Assimilating 145 proxy locations
done reconstructing:  1411
working on: 1412
Assimilating 146 proxy location

done reconstructing:  1506
working on: 1507
Assimilating 197 proxy locations
done reconstructing:  1507
working on: 1508
Assimilating 197 proxy locations
done reconstructing:  1508
working on: 1509
Assimilating 199 proxy locations
done reconstructing:  1509
working on: 1510
Assimilating 200 proxy locations
done reconstructing:  1510
working on: 1511
Assimilating 201 proxy locations
done reconstructing:  1511
working on: 1512
Assimilating 202 proxy locations
done reconstructing:  1512
working on: 1513
Assimilating 206 proxy locations
done reconstructing:  1513
working on: 1514
Assimilating 205 proxy locations
done reconstructing:  1514
working on: 1515
Assimilating 205 proxy locations
done reconstructing:  1515
working on: 1516
Assimilating 207 proxy locations
done reconstructing:  1516
working on: 1517
Assimilating 208 proxy locations
done reconstructing:  1517
working on: 1518
Assimilating 207 proxy locations
done reconstructing:  1518
working on: 1519
Assimilating 209 proxy locations

done reconstructing:  1613
working on: 1614
Assimilating 303 proxy locations
done reconstructing:  1614
working on: 1615
Assimilating 303 proxy locations
done reconstructing:  1615
working on: 1616
Assimilating 304 proxy locations
done reconstructing:  1616
working on: 1617
Assimilating 305 proxy locations
done reconstructing:  1617
working on: 1618
Assimilating 305 proxy locations
done reconstructing:  1618
working on: 1619
Assimilating 306 proxy locations
done reconstructing:  1619
working on: 1620
Assimilating 306 proxy locations
done reconstructing:  1620
working on: 1621
Assimilating 307 proxy locations
done reconstructing:  1621
working on: 1622
Assimilating 307 proxy locations
done reconstructing:  1622
working on: 1623
Assimilating 307 proxy locations
done reconstructing:  1623
working on: 1624
Assimilating 307 proxy locations
done reconstructing:  1624
working on: 1625
Assimilating 308 proxy locations
done reconstructing:  1625
working on: 1626
Assimilating 309 proxy locations

done reconstructing:  1720
working on: 1721
Assimilating 340 proxy locations
done reconstructing:  1721
working on: 1722
Assimilating 341 proxy locations
done reconstructing:  1722
working on: 1723
Assimilating 340 proxy locations
done reconstructing:  1723
working on: 1724
Assimilating 341 proxy locations
done reconstructing:  1724
working on: 1725
Assimilating 340 proxy locations
done reconstructing:  1725
working on: 1726
Assimilating 341 proxy locations
done reconstructing:  1726
working on: 1727
Assimilating 340 proxy locations
done reconstructing:  1727
working on: 1728
Assimilating 341 proxy locations
done reconstructing:  1728
working on: 1729
Assimilating 340 proxy locations
done reconstructing:  1729
working on: 1730
Assimilating 341 proxy locations
done reconstructing:  1730
working on: 1731
Assimilating 340 proxy locations
done reconstructing:  1731
working on: 1732
Assimilating 341 proxy locations
done reconstructing:  1732
working on: 1733
Assimilating 340 proxy locations

done reconstructing:  1827
working on: 1828
Assimilating 359 proxy locations
done reconstructing:  1828
working on: 1829
Assimilating 359 proxy locations
done reconstructing:  1829
working on: 1830
Assimilating 360 proxy locations
done reconstructing:  1830
working on: 1831
Assimilating 359 proxy locations
done reconstructing:  1831
working on: 1832
Assimilating 360 proxy locations
done reconstructing:  1832
working on: 1833
Assimilating 360 proxy locations
done reconstructing:  1833
working on: 1834
Assimilating 361 proxy locations
done reconstructing:  1834
working on: 1835
Assimilating 361 proxy locations
done reconstructing:  1835
working on: 1836
Assimilating 359 proxy locations
done reconstructing:  1836
working on: 1837
Assimilating 361 proxy locations
done reconstructing:  1837
working on: 1838
Assimilating 361 proxy locations
done reconstructing:  1838
working on: 1839
Assimilating 361 proxy locations
done reconstructing:  1839
working on: 1840
Assimilating 360 proxy locations

In [30]:
sic_recon = {}
sic_recon['sic_lalo'] = sic_save_lalo
sic_recon['tas_lalo'] = tas_save_lalo
sic_recon['sic_ens_var'] = np.mean(var_save)
sic_recon['nobs'] = nobs
sic_recon['sia_ens'] = np.squeeze(np.array(sic_full_ens))*nharea
sic_recon['sie_ens'] = np.squeeze(np.array(sie_full_ens))*nharea
sic_recon['recon_years'] = recon_years
sic_recon['Ye_assim'] = Ye_assim
sic_recon['Ye_assim_coords'] = Ye_assim_coords

#sic_recon['sic_ens_full'] = sic_ens_full
#sic_recon['sic_full_ens'] = sic_full_ens
#sic_recon['obs_full'] = obs_full

print('Saving to: ',savedir+savename[0])
pickle.dump(sic_recon,open(savedir+savename[0], "wb"))

Saving to:  /home/disk/p/mkb22/nobackup/LMR_output/common_era_experiments/experiments/pseudo/sic_ccsm4_anrecon_1400_1850_pseudo_inf1_loc15000_a.pkl


In [ ]:
# LOAD ANNUAL SATELLITE DATA: 
[fet_sia_anom, fet_sie_anom, fet_time] = ce_utils.load_annual_satellite_anom(2000)

In [ ]:
sie_ens_anom = sic_recon['sie_ens'] - np.nanmean(sic_recon['sie_ens'],axis=0)
sie_ensmn_anom = np.nanmean(sie_ens_anom,axis=1)

sie_97_5 = np.percentile(sie_ens_anom,97.5,axis=1)
sie_2_5 = np.percentile(sie_ens_anom,2.5,axis=1)

[ce_sie,corr_sie,var_sie_sat,var_sie_recon] = ceutils.find_ce_corr(sie_ensmn_anom, fet_sie_anom,fet_time,
                                                         np.array(sic_recon['recon_years']),1979,1999)

In [ ]:
str(cfg.core.recon_period[1])

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(sic_recon['recon_years'],sie_ensmn_anom,label='Ensemble mean',linewidth=2.5)
plt.plot(fet_time,fet_sie_anom,label='Satellite (Fetterer et al. 2017)', color='r',linewidth=2.5)
plt.fill_between(sic_recon['recon_years'],sie_2_5,sie_97_5,alpha=0.25)

plt.ylabel('Sea ice extent anomalies (10$^{6}$ $km^2$)',fontsize=14)
plt.title('Total Arctic Sea Ice Extent', fontsize=14)

plt.legend(fontsize=12)
#plt.ylim(7,13.1,1)
ce_corr_plt = (('(Satellite, DA reconstruction): R$^{2}$ = ' + '{:,.2f}'.format(corr_sie**2))+
               ', CE = '+'{:,.2f}'.format(ce_sie))
plt.gcf().text(0.14, 0.14, ce_corr_plt , fontsize=13,color='k')

#print(ce_sie,corr_sie,var_sie_sat,var_sie_recon)

savedir = ('/home/disk/p/mkb22/Documents/si_analysis_kb/common_era_experiments/'+
           'analysis/figures/sic_ccsm4_anrecon_0_2000_fullLMRdbv1_2_6_inf1_loc15000_a/')
savename = 'tot_sie_v_sat_1979_2017.png'
plt.savefig(savedir+savename)

In [ ]:
sic_anom.max()

In [ ]:
# Plot assimilated proxies: Arctic
recon_years = np.array(sic_recon['recon_years'])

sic_sat_mean = np.nanmean(sic_recon['sic_lalo'],axis=0)
sic_anom = sic_recon['sic_lalo'] - sic_sat_mean

fig,ax = plt.subplots(6,4, figsize=(20,30), subplot_kw = proj)
ax = ax.flatten()

for yr in range(recon_years.shape[0]):
    sub_arctic_plot(fig,ax[yr],sic_anom[yr,:,:],
                    grid.lat[:,0],grid.lon[0,:],str(recon_years[yr]),20, colorbar=False)

fig.suptitle('SIC Reconstructed anomalies: 1979-2000', fontsize=20)
plt.tight_layout(rect=(0,0,1,0.97))

# savedir = '/home/disk/p/mkb22/Documents/si_analysis_kb/common_era_experiments/analysis/figures/sic_ccsm4_anrecon_0_2000_pages2kv1_inf2_6_loc15000_a/'
# savename = 'sie_recon_anoms_1979_2017.png'
# plt.savefig(savedir+savename)